## FTX.US API in Python Tutorial

### Get Required Imports

In [1]:
import datetime
import requests
import pandas as pd
from client import FtxClient 
from local_settings import ftxus as settings

### Using the FTX Market API

Using the Markets API is easy. The Markets API allows us to get all of the market data. We'll be greeted with the following information when we make a request.

| Field                 | Type    | Value    | Description                                                                                                               |
| --------------------- | ------- | -------- | ------------------------------------------------------------------------------------------------------------------------- |
| name                  | string  | BTC-0628 | e.g. "BTC/USD" for spot, "BTC-PERP" for futures                                                                           |
| baseCurrency          | string  | BTC      | spot markets only                                                                                                         |
| quoteCurrency         | string  | USD      | spot markets only                                                                                                         |
| quoteVolume24h        | number  | 28914.76 |                                                                                                                           |
| change1h              | number  | 0.012    | change in the past hour                                                                                                   |
| change24h             | number  | 0.0299   | change in the past 24 hours                                                                                               |
| changeBod             | number  | 0.0156   | change since start of day (00:00 UTC)                                                                                     |
| highLeverageFeeExempt | boolean | false    |                                                                                                                           |
| minProvideSize        | number  | 0.001    | Minimum maker order size (if >10 orders per hour fall below this size)                                                    |
| type                  | string  | future   | "future" or "spot"                                                                                                        |
| underlying            | string  | BTC      | future markets only                                                                                                       |
| enabled               | boolean | true     |                                                                                                                           |
| ask                   | number  | 3949.25  | best ask                                                                                                                  |
| bid                   | number  | 3949.00  | best bid                                                                                                                  |
| last                  | number  | 3949.00  | last traded price                                                                                                         |
| postOnly              | boolean | false    | if the market is in post-only mode (all orders get modified to be post-only, in addition to other settings they may have) |
| price                 | number  | 10579.52 | current price                                                                                                             |
| priceIncrement        | number  | 0.25     |                                                                                                                           |
| sizeIncrement         | number  | 0.0001   |                                                                                                                           |
| restricted            | boolean | false    | if the market has nonstandard restrictions on which jurisdictions can trade it                                            |
| volumeUsd24h          | number  | 28914.76 | USD volume in past 24 hours                                                                                               |

### Get All Market Data

Let's start by getting all available markets. We need to use the requests library to request the appropriate API endpoint.

In [2]:
# GET /markets
api_url = 'https://ftx.us/api'
api = '/markets'
url = api_url+api
url

'https://ftx.us/api/markets'

Now we'll use requests to get the market response data.

In [3]:
markets = requests.get(url).json()
data = markets['result']
data

[{'name': 'AAVE/USD',
  'enabled': True,
  'postOnly': False,
  'priceIncrement': 0.01,
  'sizeIncrement': 0.01,
  'minProvideSize': 0.01,
  'last': 124.14,
  'bid': 124.2,
  'ask': 124.69,
  'price': 124.2,
  'type': 'spot',
  'baseCurrency': 'AAVE',
  'quoteCurrency': 'USD',
  'underlying': None,
  'restricted': False,
  'highLeverageFeeExempt': True,
  'largeOrderThreshold': 2000.0,
  'change1h': 0.03431045969353764,
  'change24h': -0.09777713206450675,
  'changeBod': -0.047619047619047616,
  'quoteVolume24h': 165801.9836,
  'volumeUsd24h': 165801.9836},
 {'name': 'AAVE/USDT',
  'enabled': True,
  'postOnly': False,
  'priceIncrement': 0.01,
  'sizeIncrement': 0.01,
  'minProvideSize': 0.01,
  'last': 124.58,
  'bid': 124.16,
  'ask': 124.55,
  'price': 124.55,
  'type': 'spot',
  'baseCurrency': 'AAVE',
  'quoteCurrency': 'USDT',
  'underlying': None,
  'restricted': False,
  'highLeverageFeeExempt': True,
  'largeOrderThreshold': 2000.0,
  'change1h': 0.037484381507705125,
  'chan

Since JSON format isn't that easy to read, let's convert it into a pandas dataframe.

In [4]:
df = pd.DataFrame(data)
df = df.set_index('name')
# using iloc to make it readable
df.iloc[:,:4].head()

,enabled,postOnly,priceIncrement,sizeIncrement
name,,,,
AAVE/USD,True,False,1.000000e-02,0.01
AAVE/USDT,True,False,1.000000e-02,0.01
AUD/USD,True,False,1.000000e-04,1.00
AVAX/BTC,True,False,1.000000e-07,0.10
AVAX/USD,True,False,5.000000e-04,0.10


Pretty easy, right? This will become second nature soon enough.

### Get an Individual Market Data

You can use the following to get a single market.

In [5]:
# GET /markets/{market_name}
market_name = 'ETH/USD'
path = f'/markets/{market_name}'
url = api_url + path
url

'https://ftx.us/api/markets/ETH/USD'

Now let's make the request and output the response as a dataframe.

In [6]:
res = requests.get(url).json()
df = pd.DataFrame(res)['result']
df

ask                             2544.2
baseCurrency                       ETH
bid                             2542.9
change1h                      0.038751
change24h                     -0.04217
changeBod                    -0.012998
enabled                           True
highLeverageFeeExempt             True
largeOrderThreshold             5000.0
last                            2543.9
minProvideSize                   0.001
name                           ETH/USD
postOnly                         False
price                           2543.9
priceIncrement                     0.1
quoteCurrency                      USD
quoteVolume24h           65957059.6477
restricted                       False
sizeIncrement                    0.001
type                              spot
underlying                        None
volumeUsd24h             65957059.6477
Name: result, dtype: object

### Get Historical Data

You can also get historical data for any of the markets. Let's get the data as daily bars starting from 2022.

In [7]:
# GET /markets/{market_name}/candles?resolution={resolution}&start_time={start_time}&end_time={end_time}

In [8]:
# Days is 60 seconds * 60 minutes * 24
resolution = 60*60*24
resolution

86400

In [9]:
start = datetime.datetime(2022,1,1).timestamp()
start

1641013200.0

We'll now construct the URL string and request as we've done above.

In [10]:
path = f'/markets/{market_name}/candles?resolution={resolution}&start={start}'
url = api_url + path
url

'https://ftx.us/api/markets/ETH/USD/candles?resolution=86400&start=1641013200.0'

And we'll put the data into the dataframe and clean up the index.

In [11]:
res = requests.get(url).json()
df = pd.DataFrame(res['result'])
df['date'] = pd.to_datetime(df['startTime'])
df = df.set_index('date')
df = df.drop(columns=['startTime', 'time'])
df.head()


,open,high,low,close,volume
date,,,,,
2020-03-23 00:00:00+00:00,130.355,136.935,129.625,136.775,0.0
2020-03-24 00:00:00+00:00,136.770,144.210,132.855,138.790,0.0
2020-03-25 00:00:00+00:00,138.785,142.980,132.750,136.115,0.0
2020-03-26 00:00:00+00:00,136.105,140.125,133.615,138.890,0.0
2020-03-27 00:00:00+00:00,138.890,141.955,129.320,131.425,0.0


## Get Order Book Data


We can also get the order book at varying depths. 

| Name        | Type   | Value    | Description                   |
| ----------- | ------ | -------- | ----------------------------- |
| market_name | string | BTC-0628 | Required. Name of the market. |
| depth       | number | 35       | max 100, default 20           |

Let's take a look at the order book for ETH/USD.

In [12]:
# GET /markets/{market_name}/orderbook?depth={depth}
depth = 20
path =  f'/markets/{market_name}/orderbook?depth={depth}'
url = api_url + path
url

'https://ftx.us/api/markets/ETH/USD/orderbook?depth=20'

In [13]:
res = requests.get(url).json()
bids = pd.DataFrame(res['result']['bids'])
asks = pd.DataFrame(res['result']['asks'])
bids.columns = ['Bid Price', 'Bid Amount']
asks.columns = ['Ask Price','Ask Amount']
bids.head()

,Bid Price,Bid Amount
0,2543.9,5.320
1,2543.6,1.520
2,2543.2,1.910
3,2542.8,1.520
4,2542.7,16.531


Let's merge the two dataframes on their range indices.

In [14]:
df = pd.merge(bids, asks, left_index=True, right_index=True)
df.head()

,Bid Price,Bid Amount,Ask Price,Ask Amount
0,2543.9,5.320,2544.2,0.001
1,2543.6,1.520,2545.0,1.520
2,2543.2,1.910,2545.3,21.176
3,2542.8,1.520,2545.6,1.520
4,2542.7,16.531,2545.7,0.120


And we can get the summary statistics using describe. Notice the depth as the count variable.

In [15]:
df.describe()

,Bid Price,Bid Amount,Ask Price,Ask Amount
count,20.000000,20.000000,20.000000,20.000000
mean,2541.250000,10.443150,2546.810000,11.709000
std,1.630144,22.722145,1.234973,31.427079
min,2538.600000,1.480000,2544.200000,0.001000
25%,2539.925000,1.520000,2545.850000,1.520000
50%,2541.350000,1.520000,2546.950000,1.520000
75%,2542.550000,6.026750,2547.825000,3.353000
max,2543.900000,98.275000,2548.500000,139.181000


Now let's merge the two dataframes and rename the columns.

In [16]:
df = pd.merge(bids, asks, left_index=True, right_index=True)
df.head()

,Bid Price,Bid Amount,Ask Price,Ask Amount
0,2543.9,5.320,2544.2,0.001
1,2543.6,1.520,2545.0,1.520
2,2543.2,1.910,2545.3,21.176
3,2542.8,1.520,2545.6,1.520
4,2542.7,16.531,2545.7,0.120


## Get Trades
Getting the recent trades for any market is also easy.

| Name        | Type   | Value      | Description        |
| ----------- | ------ | ---------- | ------------------ |
| market_name | string | BTC-0628   | name of the market |
| start_time  | number | 1559881511 | optional           |
| end_time    | number | 1559881711 | optional           |


In [17]:
# GET /markets/{market_name}/trades
path = f'/markets/{market_name}/trades'
url = api_url + path
url


'https://ftx.us/api/markets/ETH/USD/trades'

In [18]:
res = requests.get(url).json()
df = pd.DataFrame(res['result'])
df.head()

,id,price,size,side,liquidation,time
0,25598828,2543.9,1.520,sell,False,2022-02-24T19:20:18.583827+00:00
1,25598812,2542.7,0.001,buy,False,2022-02-24T19:20:14.499667+00:00
2,25598787,2539.0,0.001,buy,False,2022-02-24T19:19:53.365086+00:00
3,25598776,2539.2,0.002,sell,False,2022-02-24T19:19:39.537406+00:00
4,25598764,2537.9,0.002,buy,False,2022-02-24T19:19:31.126433+00:00


## Working with FTX Futures

Now that you've learned how to use the Markets API, using the Futures API is just as easy. I'll get you started. One thing to note is that /futures is not available through the ftx.us API.

First, check out the [futures API documenation](https://docs.ftx.com/#futures). You'll notice we need to make a get request to /futures. Let's create the request URL and use requests to get the data.

In [19]:
url = 'https://ftx.com/api/futures'
res = requests.get(url).json()
res


{'success': True,
 'result': [{'name': '1INCH-PERP',
   'underlying': '1INCH',
   'description': '1INCH Token Perpetual Futures',
   'type': 'perpetual',
   'expiry': None,
   'perpetual': True,
   'expired': False,
   'enabled': True,
   'postOnly': False,
   'priceIncrement': 0.0001,
   'sizeIncrement': 1.0,
   'last': 1.2989,
   'bid': 1.2983,
   'ask': 1.3007,
   'index': 1.2971271776125,
   'mark': 1.2983,
   'imfFactor': 0.0005,
   'lowerBound': 1.2311,
   'upperBound': 1.3619,
   'underlyingDescription': '1INCH Token',
   'expiryDescription': 'Perpetual',
   'moveStart': None,
   'marginPrice': 1.2983,
   'positionLimitWeight': 20.0,
   'group': 'perpetual',
   'change1h': 0.024946711928633458,
   'change24h': -0.09576542693968519,
   'changeBod': -0.061175790006508066,
   'volumeUsd24h': 9239002.5818,
   'volume': 7307335.0,
   'openInterest': 7123256.0,
   'openInterestUsd': 9248123.2648},
  {'name': '1INCH-0325',
   'underlying': '1INCH',
   'description': '1INCH Token March 

In [20]:
df = pd.DataFrame(res['result'])
# Using iloc to make it readable
df.iloc[:,:4].head()

,name,underlying,description,type
0,1INCH-PERP,1INCH,1INCH Token Perpetual Futures,perpetual
1,1INCH-0325,1INCH,1INCH Token March 2022 Futures,future
2,AAPL-0325,AAPL,Apple March 2022 Futures,future
3,AAVE-PERP,AAVE,Aave Perpetual Futures,perpetual
4,AAVE-0325,AAVE,Aave March 2022 Futures,future
